In [ ]:
import os, gc, sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.linalg import cholesky
from statsmodels.tsa.arima_process import ArmaProcess
import matplotlib.pyplot as plt

In [ ]:
def generate_mixture_lognormal(means, variances, size):
    if isinstance(means, list): means = np.array(means)
    if isinstance(variances, list): variances = np.array(variances)
    if isinstance(means, (int, float)): means = np.array([float(means) for _ in range(variances.shape[0])])
    if isinstance(variances, (int, float)): variances = np.array([float(variances) for _ in range(means.shape[0])])    
    assert means.shape==variances.shape, "Mismatch in length of means and variances"
    # covert means and variances of lognormal to those of normal
    means_normal = np.log(means) - 0.5 * np.log(variances/means**2+1)
    variances_normal = np.log(variances/means**2+1)
    idx = np.random.randint(0, np.shape(means_normal), size)
    means_array = np.array(means_normal)[idx]
    stdvs_array = np.sqrt(np.array(variances_normal)[idx])    
    data = np.exp(np.random.normal(means_array, stdvs_array))
    return data

In [ ]:
def generate_compound_symmetry(sample_size, n_features, rho=0.5, sigma=1.0):
    cov_matrix = sigma**2 * ((1-rho) * np.eye(n_features) + rho * np.ones((n_features, n_features)))
    L = cholesky(cov_matrix, lower=True)
    X = np.random.normal(0, 1, (sample_size, n_features)) @ L.T
    return X
def generate_mixed_ar_cs(sample_size, n_features, ar=0.5, rho=0.5, sigma=1.0):
    # Generate a mixture of AR(1) and Compound Symmetry data with binomially determined proportions
    n_ar = np.random.binomial(sample_size, 0.5)  # Random proportion of AR(1) samples
    n_cs = sample_size - n_ar
    arma = ArmaProcess(ar=[1, -ar], ma=1)
    X_ar = arma.generate_sample(nsample=(n_ar, n_features), axis=1)
    X_cs = generate_compound_symmetry(n_cs, n_features, rho, sigma)    
    X_mixed = np.vstack((X_ar, X_cs))
    np.random.shuffle(X_mixed)  # Shuffle to mix AR(1) and CS samples
    return X_mixed

In [ ]:
def generate_mean(beta, input_distribution, random_effects_distribution, random_effects_variance, n_clusters, cluster_size):    
    n_features = 50
    sample_size = n_clusters * cluster_size
    beta = beta.reshape(-1,1)    
    # generate X -------------------------------------------------------
    if input_distribution.upper() == 'IND': # independent
        X = np.random.normal(0, 1, (sample_size, 100))
    elif input_distribution.upper() == 'AR1':
        arma = ArmaProcess(ar=[1, -0.5], ma=1)
        X = arma.generate_sample(nsample=(sample_size, 100), axis=1)
    elif input_distribution.upper() == 'AR2':
        arma = ArmaProcess(ar=[1, -0.5, -0.25], ma=1)
        X = arma.generate_sample(nsample=(sample_size, 100), axis=1)
    elif input_distribution.upper() == 'SYM': # compound symmetry
        X = generate_compound_symmetry(sample_size, 100, rho=0.5)
    elif input_distribution.upper() == 'MIX': # mixture of compound symmetry and AR1
        X = generate_mixed_ar_cs(sample_size, 100, ar=0.5, rho=0.5)
    else:
        X = np.random.uniform(-np.pi/2, np.pi/2, (sample_size, 100))
    data = pd.DataFrame(X, columns=[('x'+str(i)) for i in range(n_features)])
    data['cluster'] = np.repeat(np.arange(n_clusters), cluster_size)
    data['number'] = np.tile(np.arange(cluster_size), n_clusters)    
    # generate u -------------------------------------------------------
    if random_effects_distribution == 'X':
        u = np.repeat(1., n_clusters)
    elif random_effects_distribution == 'G': 
        u = np.random.gamma(1./random_effects_variance, random_effects_variance, n_clusters)
    elif random_effects_distribution == 'N':
        sig2 = np.log((1+np.sqrt(1+4*random_effects_variance))/2)
        u = np.exp(np.random.normal(0., np.sqrt(sig2), n_clusters))        
    elif random_effects_distribution == 'M':
        means = [0.5, 1.5]
        variances = [(4.*random_effects_variance-1.)/20., 9.*(4.*random_effects_variance-1.)/20.]
        u = generate_mixture_lognormal(means, variances, n_clusters)
    else: raise ValueError(f"Unknown random effects distribution: {random_effects_distribution}")
    data['u'] = np.repeat(u, cluster_size)
    # generate mu ------------------------------------------------------
    data['mu']=np.array(data['u']).reshape(-1,1)*np.exp(
        np.sin(X[:,:10])@beta + np.sin(X[:,20:30]*X[:,30:40])@beta
        + np.cos(X[:,10:20])@beta + np.cos(X[:,30:40]*X[:,40:50])@beta
    )
    # ------------------------------------------------------------------
    return data

In [ ]:
def generate_count_data(path, beta, input_distributions, random_effects_types, n_clusters, cluster_size, n_repeats):
    sample_size = n_clusters*cluster_size
    for random_effects_type in random_effects_types:
        random_effects_distribution, random_effects_variance = random_effects_type.split('-')
        for input_distribution in input_distributions:
            for simulation_number in tqdm(range(n_repeats), desc=f'X: {input_distribution}, u: {random_effects_type}'):
                np.random.seed(simulation_number)
                data = generate_mean(beta, input_distribution, random_effects_distribution, float(random_effects_variance), n_clusters, cluster_size)
                data['y'] = np.random.poisson(data['mu'], data['mu'].shape)                
                file_name = f'{path}/data-{input_distribution}-{random_effects_type}-{simulation_number}.csv'
                data.to_csv(file_name, index=False)
                del data; gc.collect()

In [ ]:
path_data = os.path.abspath('data')
n_repeats = 10
n_clusters, cluster_size = 10000, 12
# ------------------------------------------------------------------
beta = 0.02*np.array(list(np.arange(-5,0)) + list(np.arange(1,6)))
# ------------------------------------------------------------------
input_distributions = ['IND', 'AR1', 'AR2', 'SYM', 'MIX']
random_effects_types = ['X-0.0']
for random_effects_distribution in ['G', 'N', 'M']: # gamma, log-normal, mixture
    for random_effects_variance in [1.0, 2.0]:
        random_effects_types.append(f'{random_effects_distribution}-{random_effects_variance:.1f}')
generate_count_data(path_data, beta, input_distributions, random_effects_types, n_clusters, cluster_size, n_repeats)

In [ ]:
sys.path.append(os.path.abspath(os.path.join('..', 'source')))
from metrics import compute_metrics

In [ ]:
path_res  = os.path.abspath('results')
n_clusters, cluster_size = 10000, 12
cluster_size_train, cluster_size_valid, cluster_size_test = 8, 2, 2
metrics = ['RMSP', 'RMD', 'R2']

In [ ]:
# ------------------------------------------------------------------
model_name = 'true'
# ------------------------------------------------------------------
results = {metric:np.zeros((n_repeats*len(input_distributions), len(random_effects_types))) for metric in metrics+['time']}
experiments = [f'{input_distribution}-{simulation_number}' for input_distribution in input_distributions for simulation_number in range(n_repeats)]
for k_re, random_effects_type in enumerate(random_effects_types):    
    for k_in, input_distribution in enumerate(input_distributions):
        for simulation_number in tqdm(range(n_repeats), desc=f'X: {input_distribution}, u: {random_effects_type}'):
            data = pd.read_csv(f'{path_data}/data-{input_distribution}-{random_effects_type}-{simulation_number}.csv')
            data_test = data[data['number'].isin(range(cluster_size_train+cluster_size_valid, cluster_size))]
            y_test = np.array(data_test['y'],  dtype=np.float32)
            y_pred = np.array(data_test['mu'], dtype=np.float32)
            temp_results = compute_metrics(y_test, y_pred, metrics)
            for metric in metrics:
                results[metric][(k_in*n_repeats+simulation_number), k_re] = temp_results[metric]
            results['time'][(k_in*n_repeats+simulation_number), k_re] = 0.
    for metric in metrics+['time']:
        print(f'{metric.upper()}: {"{:.3f}".format(np.mean(results[metric][:, k_re]))} ({"{:.3f}".format(np.std(results[metric][:, k_re]))})')
for metric in metrics+['time']:
    pd.DataFrame(results[metric], columns=random_effects_types, index=experiments).to_csv(f'{path_res}/{model_name}-{metric}.csv', index=True)